In [1]:
import pandas as pd
import QUANTAXIS as qtx
from datetime import datetime, timedelta

In [14]:
block = pd.read_csv("c:\\Users\\mUSicX\\Documents\\股票相关\\20200922SW.csv")
block['c'] = block.code.apply(lambda x: x[:6])
block['mkt'] = block.code.apply(lambda x: 0 if x.endswith('SZ') else 1)
block['cat1'] = block.ind.apply(lambda x: x.split('--')[0])
block['cat2'] = block.ind.apply(lambda x: '-'.join(x.split('--')[:2]))
block['cat3'] = block.ind.apply(lambda x: '-'.join(x.split('--')[:3]))
#raw['ind_cat3'] = raw.ind.apply(lambda x: x.split('--')[2].replace('Ⅲ', ''))

#raw['ind_cat'] = raw.apply(lambda x: x.ind_cat1+'-'+x.ind_cat2, axis=1)
block = block.loc[:, ['c', 'cat1', 'cat2', 'cat3']]
block.rename(columns={'cat2':'cat'}, inplace=True)
block[:10]

,c,cat1,cat,cat3
0,000009,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
1,000409,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
2,000632,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
3,000638,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
4,000652,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
5,000839,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
6,002967,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
7,300012,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
8,300688,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ
9,300797,综合,综合-综合Ⅱ,综合-综合Ⅱ-综合Ⅲ


In [3]:
code_list = qtx.QA_fetch_stock_list()
finfo =  qtx.QA_fetch_stock_info(list(code_list.code))
fcnd = qtx.QA_fetch_stock_day_adv(list(code_list.code),  #['000001', '601555', '601058'],  
                                  start=(datetime.today()-timedelta(days=366)).strftime('%Y-%m-%d'), 
                                  end=datetime.today().strftime('%Y-%m-%d')).to_qfq()
fcnd.data.head(10)

open       high        low      close    volume  \
date       code                                                           
2019-10-15 000001  16.960731  17.059054  16.537942  16.891905  881955.0   
           000002  26.223507  26.310308  25.741280  25.818436  500297.0   
           000004  19.080000  19.080000  18.750000  18.840000    4640.0   
           000005   3.290000   3.540000   3.280000   3.350000  324570.0   
           000006   5.066319   5.114847   5.008086   5.066319   57875.0   
           000007  10.450000  10.450000   9.460000   9.640000  233337.0   
           000008   3.587879   3.627744   3.538047   3.577912  394369.0   
           000009   4.747949   4.787848   4.698076   4.747949  204045.0   
           000010   3.440000   3.540000   3.420000   3.440000   19036.0   
           000011   9.510492   9.933180   9.318361   9.769869   39360.0   

                         amount       adj  
date       code                            
2019-10-15 000001  1.503633e+09  0.983231  
           000002  1.346872e+09  0.964454  
           000004  8.754233e+06  1.000000  
           000005  1.102731e+08  1.000000  
           000006  3.011554e+07  0.970559  
           000007  2.329616e+08  1.000000  
           000008  1.415561e+08  0.996633  
           000009  9.705717e+07  0.997468  
           000010  6.591041e+06  1.000000  
           000011  3.946717e+07  0.960656

In [4]:
fbase = fcnd.data.join(finfo.loc[finfo.ipo_date != 0, ['ipo_date','industry']], how='inner').reset_index()
fbase['ipodays'] = fbase.apply(lambda x: (x.date - datetime.strptime(str(x.ipo_date), '%Y%m%d')).days, axis=1)
fbase.head(10)

,date,code,open,high,low,close,volume,amount,adj,ipo_date,industry,ipodays
0,2019-10-15,000001,16.960731,17.059054,16.537942,16.891905,881955.0,1.503633e+09,0.983231,19910403,1,10422
1,2019-10-15,000002,26.223507,26.310308,25.741280,25.818436,500297.0,1.346872e+09,0.964454,19910129,11,10486
2,2019-10-15,000004,19.080000,19.080000,18.750000,18.840000,4640.0,8.754233e+06,1.000000,19910114,34,10501
3,2019-10-15,000005,3.290000,3.540000,3.280000,3.350000,324570.0,1.102731e+08,1.000000,19901210,39,10536
4,2019-10-15,000006,5.066319,5.114847,5.008086,5.066319,57875.0,3.011554e+07,0.970559,19920427,11,10032
5,2019-10-15,000007,10.450000,10.450000,9.460000,9.640000,233337.0,2.329616e+08,1.000000,19920413,10,10046
6,2019-10-15,000008,3.587879,3.627744,3.538047,3.577912,394369.0,1.415561e+08,0.996633,19920507,8,10022
7,2019-10-15,000009,4.747949,4.787848,4.698076,4.747949,204045.0,9.705717e+07,0.997468,19910625,25,10339
8,2019-10-15,000010,3.440000,3.540000,3.420000,3.440000,19036.0,6.591041e+06,1.000000,19951027,21,8754
9,2019-10-15,000011,9.510492,9.933180,9.318361,9.769869,39360.0,3.946717e+07,0.960656,19920330,11,10060


In [15]:
guben = finfo['zongguben'].reset_index()
fgb = pd.merge(fbase.loc[fbase['ipodays'] > 120,:], guben, on='code')
fgb.sort_values(['code', 'date'], inplace=True)
fgb['close_sz'] = fgb['close'] * fgb['zongguben']
fgb['change'] = fgb['close'] / fgb['close'].shift(1) - 1
fgb = pd.merge(fgb.loc[:, ['date', 'code', 'close_sz', 'change']], block.loc[:, ['c', 'cat']], left_on='code', right_on='c')
fgb.head(20)

,date,code,close_sz,change,c,cat
0,2019-10-15,000001,3.278029e+11,NaN,000001,银行-银行
1,2019-10-16,000001,3.203615e+11,-0.022701,000001,银行-银行
2,2019-10-17,000001,3.186443e+11,-0.005360,000001,银行-银行
3,2019-10-18,000001,3.150190e+11,-0.011377,000001,银行-银行
4,2019-10-21,000001,3.222696e+11,0.023016,000001,银行-银行
5,2019-10-22,000001,3.133018e+11,-0.027827,000001,银行-银行
6,2019-10-23,000001,3.138742e+11,0.001827,000001,银行-银行
7,2019-10-24,000001,3.218880e+11,0.025532,000001,银行-银行
8,2019-10-25,000001,3.220788e+11,0.000593,000001,银行-银行
9,2019-10-28,000001,3.178811e+11,-0.013033,000001,银行-银行


In [16]:
block_size = fgb.groupby(['cat', 'date'])['close_sz'].sum().reset_index()
block_size.rename(columns={'close_sz':'close_all'}, inplace=True)
block_size.head(10)

,cat,date,close_all
0,交通运输-公交Ⅱ,2019-10-15,3.629577e+10
1,交通运输-公交Ⅱ,2019-10-16,3.599953e+10
2,交通运输-公交Ⅱ,2019-10-17,3.583743e+10
3,交通运输-公交Ⅱ,2019-10-18,3.539409e+10
4,交通运输-公交Ⅱ,2019-10-21,3.530210e+10
5,交通运输-公交Ⅱ,2019-10-22,3.565810e+10
6,交通运输-公交Ⅱ,2019-10-23,3.544749e+10
7,交通运输-公交Ⅱ,2019-10-24,3.539270e+10
8,交通运输-公交Ⅱ,2019-10-25,3.568148e+10
9,交通运输-公交Ⅱ,2019-10-28,3.625152e+10


In [17]:
bkw = pd.merge(fgb, block_size, on=['cat', 'date'])
bkw['weight'] = bkw.close_sz / bkw.close_all
bkw.head(10)

,date,code,close_sz,change,c,cat,close_all,weight
0,2019-10-15,000001,3.278029e+11,NaN,000001,银行-银行,1.009679e+13,0.032466
1,2019-10-15,002142,1.603811e+11,7.781093,002142,银行-银行,1.009679e+13,0.015884
2,2019-10-15,002807,9.639869e+09,-0.634410,002807,银行-银行,1.009679e+13,0.000955
3,2019-10-15,002839,1.029458e+10,-0.812780,002839,银行-银行,1.009679e+13,0.001020
4,2019-10-15,002936,2.821573e+10,-0.830405,002936,银行-银行,1.009679e+13,0.002795
5,2019-10-15,002948,2.873585e+10,-0.882021,002948,银行-银行,1.009679e+13,0.002846
6,2019-10-15,002958,3.777365e+10,-0.724280,002958,银行-银行,1.009679e+13,0.003741
7,2019-10-15,600000,3.643800e+11,-0.975122,600000,银行-银行,1.009679e+13,0.036089
8,2019-10-15,600015,1.148631e+11,0.435546,600015,银行-银行,1.009679e+13,0.011376
9,2019-10-15,600016,2.579043e+11,-0.055995,600016,银行-银行,1.009679e+13,0.025543


In [18]:
fbkw = pd.merge(fbase.reset_index(), bkw.loc[:, ['date', 'code', 'cat', 'weight']], on=['date', 'code'])
fbkw.head(10)

,index,date,code,open,high,low,close,volume,amount,adj,ipo_date,industry,ipodays,cat,weight
0,0,2019-10-15,000001,16.960731,17.059054,16.537942,16.891905,881955.0,1.503633e+09,0.983231,19910403,1,10422,银行-银行,0.032466
1,1,2019-10-15,000002,26.223507,26.310308,25.741280,25.818436,500297.0,1.346872e+09,0.964454,19910129,11,10486,房地产-房地产开发Ⅱ,0.155974
2,2,2019-10-15,000004,19.080000,19.080000,18.750000,18.840000,4640.0,8.754233e+06,1.000000,19910114,34,10501,医药生物-化学制药,0.002271
3,3,2019-10-15,000005,3.290000,3.540000,3.280000,3.350000,324570.0,1.102731e+08,1.000000,19901210,39,10536,公用事业-环保工程及服务,0.011285
4,4,2019-10-15,000006,5.066319,5.114847,5.008086,5.066319,57875.0,3.011554e+07,0.970559,19920427,11,10032,房地产-房地产开发Ⅱ,0.003557
5,5,2019-10-15,000007,10.450000,10.450000,9.460000,9.640000,233337.0,2.329616e+08,1.000000,19920413,10,10046,商业贸易-商业物业经营,0.027201
6,6,2019-10-15,000008,3.587879,3.627744,3.538047,3.577912,394369.0,1.415561e+08,0.996633,19920507,8,10022,机械设备-运输设备,0.035907
7,7,2019-10-15,000009,4.747949,4.787848,4.698076,4.747949,204045.0,9.705717e+07,0.997468,19910625,25,10339,综合-综合Ⅱ,0.077794
8,8,2019-10-15,000010,3.440000,3.540000,3.420000,3.440000,19036.0,6.591041e+06,1.000000,19951027,21,8754,建筑装饰-园林工程,0.031447
9,9,2019-10-15,000011,9.510492,9.933180,9.318361,9.769869,39360.0,3.946717e+07,0.960656,19920330,11,10060,房地产-房地产开发Ⅱ,0.003028


In [19]:
fbkw['wopen'] = fbkw.open * fbkw.weight
fbkw['wclose'] = fbkw.close * fbkw.weight
fbkw['whigh'] = fbkw.high * fbkw.weight
fbkw['wlow'] = fbkw.low * fbkw.weight
block_group = fbkw.groupby(['date', 'cat'])['wopen', 'wclose', 'whigh', 'wlow'].sum()
block_group.head(10)

wopen     wclose      whigh       wlow
date       cat                                                    
2019-10-15 交通运输-公交Ⅱ      8.355187   8.462738   8.515587   8.194377
           交通运输-机场Ⅱ     58.296641  57.992154  58.562774  56.875090
           交通运输-港口Ⅱ      5.943787   5.941038   6.001579   5.879428
           交通运输-物流Ⅱ     21.881453  22.327827  22.483431  21.666563
           交通运输-航空运输Ⅱ   10.530700  10.602751  10.677257  10.463495
           交通运输-航运Ⅱ      5.374827   5.236756   5.476115   5.164271
           交通运输-铁路运输Ⅱ    6.326350   6.312991   6.338999   6.285467
           交通运输-高速公路Ⅱ    6.887384   6.888959   6.979343   6.837097
           休闲服务-其他休闲服务  17.618358  17.733258  17.868834  17.400470
           休闲服务-旅游综合Ⅱ   77.450866  77.993088  78.160199  76.988976

In [20]:
block_group.reset_index(inplace=True)
block_group.sort_values(['cat','date'], inplace=True, ascending=[True, False])
block_group['goup'] = block_group.wclose / block_group.wopen
block_group['raise'] = block_group.wclose / block_group.wclose.shift(-1)
block_group.head(10)

,date,cat,wopen,wclose,whigh,wlow,goup,raise
25272,2020-10-15,交通运输-公交Ⅱ,11.861431,11.661635,12.033525,11.499012,0.983156,0.968362
25168,2020-10-14,交通运输-公交Ⅱ,12.048006,12.042643,12.399809,11.760471,0.999555,1.007242
25064,2020-10-13,交通运输-公交Ⅱ,12.061317,11.956053,12.140113,11.625009,0.991273,1.003813
24960,2020-10-12,交通运输-公交Ⅱ,11.790068,11.910634,12.048894,11.571717,1.010226,1.024841
24856,2020-10-09,交通运输-公交Ⅱ,11.287942,11.621935,11.740617,11.189644,1.029588,1.046603
24752,2020-09-30,交通运输-公交Ⅱ,11.234106,11.104439,11.459121,10.923629,0.988458,0.979961
24648,2020-09-29,交通运输-公交Ⅱ,11.603998,11.331515,11.824646,11.123853,0.976518,0.961180
24544,2020-09-28,交通运输-公交Ⅱ,12.548449,11.789170,12.844522,11.755407,0.939492,0.912986
24440,2020-09-25,交通运输-公交Ⅱ,12.913351,12.912764,13.609084,12.611147,0.999955,1.001187
24336,2020-09-24,交通运输-公交Ⅱ,12.462826,12.897450,13.061521,12.303874,1.034874,1.059873


In [21]:
pvt = pd.pivot(block_group, index='cat', columns='date', values='raise')
columns = pvt.columns.tolist()
columns = columns[::-1]
pvt = pvt[columns]
pvt.head(10)

date,2020-10-15 00:00:00,2020-10-14 00:00:00,2020-10-13 00:00:00,2020-10-12 00:00:00,2020-10-09 00:00:00,2020-09-30 00:00:00,2020-09-29 00:00:00,2020-09-28 00:00:00,2020-09-25 00:00:00,2020-09-24 00:00:00,...,2019-10-28 00:00:00,2019-10-25 00:00:00,2019-10-24 00:00:00,2019-10-23 00:00:00,2019-10-22 00:00:00,2019-10-21 00:00:00,2019-10-18 00:00:00,2019-10-17 00:00:00,2019-10-16 00:00:00,2019-10-15 00:00:00
cat,,,,,,,,,,,,,,,,,,,,,
交通运输-公交Ⅱ,0.968362,1.007242,1.003813,1.024841,1.046603,0.979961,0.961180,0.912986,1.001187,1.059873,...,1.011638,1.006499,0.998210,0.994564,1.005335,0.995780,0.980567,0.991120,0.977072,0.157437
交通运输-机场Ⅱ,0.985510,1.013452,0.998383,1.006647,1.008969,0.995118,1.013772,0.989926,1.002869,0.998365,...,0.997068,1.013086,0.970318,0.999187,0.992381,0.977880,0.986964,1.031319,0.999887,11.052447
交通运输-港口Ⅱ,0.988234,0.977915,1.002175,1.026726,1.035495,0.994731,1.004921,0.993625,0.985712,0.981232,...,1.005166,1.011538,1.003222,0.989499,1.004348,0.996056,0.978816,0.999813,0.991711,0.111760
交通运输-物流Ⅱ,0.963989,1.023491,1.058670,1.011484,1.061944,1.017463,1.003842,1.042141,1.012861,0.980009,...,1.006286,1.015026,0.976439,0.957917,0.999466,1.033647,0.994810,0.995447,1.005200,2.178377
交通运输-航空运输Ⅱ,0.998509,0.970118,1.018582,1.009801,0.987610,1.000015,0.993979,1.008033,1.003321,0.965121,...,0.997232,0.998877,0.990975,0.992750,1.009715,1.009378,0.996045,1.006903,0.995634,1.829465
交通运输-航运Ⅱ,0.984660,1.001704,0.994455,1.016486,1.039342,0.980921,1.016698,0.998555,0.985037,0.973609,...,1.000627,1.001121,0.997628,0.983728,1.007917,0.991368,0.976159,0.962984,0.963894,0.907983
交通运输-铁路运输Ⅱ,0.992068,0.996020,0.994300,1.009684,1.004585,1.002027,0.998753,0.987129,0.985059,0.991794,...,0.999303,0.999348,0.994714,1.005949,1.007854,0.985182,0.999344,0.997137,1.006467,0.959584
交通运输-高速公路Ⅱ,0.998029,0.984915,0.998538,1.011522,1.013321,0.996676,1.005973,0.994985,0.997050,0.990031,...,1.002157,1.003791,0.995765,0.993039,1.005986,1.006106,0.989866,1.001972,1.001533,0.164372
休闲服务-其他休闲服务,1.006814,1.032668,0.980405,0.979437,1.015309,1.037213,1.017216,1.020557,1.024221,0.960288,...,1.000710,0.987302,1.014691,0.991500,1.033445,0.997084,0.995429,1.029950,0.986563,0.098181


In [22]:
pvt.to_excel('cat_curve.xls')

In [ ]:
fbkw.loc[fbkw.cat3=='银行-银行-银行', :]

In [ ]:
block_group.loc[block_group.cat3=='银行-银行-银行', :]

In [ ]:
test = fbkw.loc[fbkw.cat3=='银行-银行-银行', :]
test.to_excel('test.xls')

In [ ]:
tcnd = qtx.QA_fetch_stock_day_adv(['601288', '600919', '601997'],  
                                  start=(datetime.today()-timedelta(days=1)).strftime('%Y-%m-%d'), 
                                  end=datetime.today().strftime('%Y-%m-%d')).to_qfq()
tcnd.data